# Rango Consumos por Persona y Diagnósticos

Se calcula con consumos por período de prestación en un reporte y stock en otro reporte

In [14]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import datetime
import funciones

In [15]:
# DEFINICION DE LAS RUTAS Y VARIABLES:

# Archivo al que se exportara la informacion:
archivo_diagnosticos = 'C:/modelos/Aux_2021/dx_rosario_2111.csv'

# Rutas donde se encuentran los 12 meses de consumo y stock por persona:
ruta_cons_RC = "C:/Users/lsaucedo/Desktop/HISTORICO ROSARIO/202111/Consumo_12M/"
ruta_stock_RC = "C:/Users/lsaucedo/Desktop/HISTORICO ROSARIO/202111/Stock_12M/"

# Ruta con los 12 meses de stock de SanCor por persona:
ruta_stock_ss = "C:/Users/lsaucedo/Desktop/HISTORICO ROSARIO/202111/Stock_posicion/"


# Defino importe gran consumidor
importe_gc = 693643
# De cada rango se define su límite inferior para consumo mensual
c_muy_alto = importe_gc / 12
c_alto = 0.015714129 * importe_gc # el monto se actualiza de acuerdo al importe gran consumidor
c_medio = 0.000785706 * importe_gc # el monto se actualiza de acuerdo al importe gran consumidor
c_bajo = 0.000314283 * importe_gc # el monto se actualiza de acuerdo al importe gran consumidor
# El consumo c_muy_bajo queda por defecto.


# proporcion limite para ser considerado persona con diagnostico
prop_limite = 0.4 



In [16]:
# Importamos los datos de consumo por periodo consumo y stock por periodo
dfc_RC = funciones.leer_xlsx(ruta_cons_RC)
dfs_RC = funciones.leer_xlsx(ruta_stock_RC)

Inicio del proceso: 01:28:53
Se cargó el archivo:  RC 202111.xlsx a las: 01:29:45
Fin de la carga de archivos
Inicio del proceso: 01:29:45
Se cargó el archivo:  202111.xlsx a las: 01:30:28
Fin de la carga de archivos


In [17]:
# renombramos las columnas de ambos dataframes
funciones.renombrar_columnas(dfs_RC)
funciones.renombrar_columnas(dfc_RC)

In [18]:
# Agregamos a dfs el consumo de dfc:
dfs_RC = pd.merge(left = dfs_RC, right = dfc_RC, left_on='Persona', right_on='Persona', how = "left")
# Eliminamos el dfc de la memoria:
del dfc_RC

In [19]:
# Marco los GC en el dataframe de stock
dfs_RC['GC'] = np.where(dfs_RC['Consumo'] > importe_gc, 'GC','NO GC')

# Marco Rango de Consumos:
# Cálculo de consumo anualizado para rango consumos:
dfs_RC['Cons_mensualizado'] = (dfs_RC['Consumo'] / dfs_RC['Stock'])

# Como existen personas con stock == 0 -> reemplazo los nan con el valor de la columna "Consumo"
# No encontré una función -> hago una reimputación manual
for i in range(len(dfs_RC)):
    if np.isnan(dfs_RC["Cons_mensualizado"][i]) == True:
        dfs_RC["Cons_mensualizado"][i] = dfs_RC["Consumo"][i]

condiciones = [
    (dfs_RC['Cons_mensualizado'] >= c_muy_alto) ,
    (dfs_RC['Cons_mensualizado'] >= c_alto) ,
    (dfs_RC['Cons_mensualizado'] >= c_medio) ,
    (dfs_RC['Cons_mensualizado'] >= c_bajo) ,
    (dfs_RC['Cons_mensualizado'] < c_bajo)]

# Lista de resultados en función de las selecciones
valores_condic = ['Muy Alto', 'Alto', 'Medio', 'Bajo', 'Muy Bajo']     

# Generación de la columna calculada
dfs_RC['Rango_Cons'] = np.select(condiciones, valores_condic, default='Muy Bajo') # se analizaron y surgen de consumo 3 == 0


C:\Users\lsaucedo\AppData\Local\Temp/ipykernel_15540/832951618.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_RC["Cons_mensualizado"][i] = dfs_RC["Consumo"][i]


In [20]:
# OPCIONAL - Exportar el dataframe a csv para tener las marcas a futuro
#dfs_RC.to_csv("Rango_cons.csv", encoding = 'latin1', decimal=',', index = False)

In [21]:
## IMPORTAMOS EL DATAFRAME CON TODO EL STOCK PARA MARCAR LOS DIAGNOSTICOS
dfs_ss = funciones.leer_xlsx(ruta_stock_ss)
funciones.renombrar_columnas(dfs_ss)


Inicio del proceso: 01:31:06
Se cargó el archivo:  S 202012.xlsx a las: 01:31:17
Se cargó el archivo:  S 202101.xlsx a las: 01:31:28
Se cargó el archivo:  S 202102.xlsx a las: 01:31:39
Se cargó el archivo:  S 202103.xlsx a las: 01:31:50
Se cargó el archivo:  S 202104.xlsx a las: 01:32:01
Se cargó el archivo:  S 202105.xlsx a las: 01:32:11
Se cargó el archivo:  S 202106.xlsx a las: 01:32:21
Se cargó el archivo:  S 202107.xlsx a las: 01:32:29
Se cargó el archivo:  S 202108.xlsx a las: 01:32:37
Se cargó el archivo:  S 202109.xlsx a las: 01:32:44
Se cargó el archivo:  S 202110.xlsx a las: 01:32:52
Se cargó el archivo:  S 202111.xlsx a las: 01:32:59
Fin de la carga de archivos


In [22]:
# Pegamos a dfs_ss el rango de consumo:
dfs_ss = pd.merge(left = dfs_ss, right = dfs_RC[['Persona', 'GC', 'Rango_Cons']], left_on='Persona', right_on='Persona', how = "left")

In [23]:
# Existen casos con nan a reemplazar (decision leo)
# Al ser personas encontradas en el stock y no encontradas en el consumo ponemos: No GC y Rango Cons == Muy Bajo
values = {"GC": "NO GC", "Rango_Cons": "Muy Bajo"}
dfs_ss = dfs_ss.fillna(value=values)

In [24]:
# Transformaciones necesarias para el calculo de los diagnosticos como metricas:
dfs_ss['GC'] = dfs_ss['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs_ss['Diabetes'] = dfs_ss['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs_ss['Hepatitis'] = dfs_ss['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs_ss['PMI'] = dfs_ss['PMI'].replace(['NO', 'SI'], [0, 1])
dfs_ss['Cronico'] = dfs_ss['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs_ss['Oncologico'] = dfs_ss['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs_ss['GC'] = dfs_ss['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs_ss['Discapacidad'] = dfs_ss['Discapacidad'].replace(['N', 'S'], [0, 1])


# Creo una columna de resultado que marque 1 si se tiene cualquier diagnostico
suma = (dfs_ss['Diabetes'] + dfs_ss['Hepatitis'] + dfs_ss['PMI'] + 
        dfs_ss['Cronico'] + dfs_ss['Oncologico'] + dfs_ss['GC'] + dfs_ss['Discapacidad'])

dfs_ss['algun_diag'] = np.where( suma > 0, 1, 0)

In [25]:
# Creo una auxiliar que resuma por persona el stock y cantidad de veces con diag.
# Agrupo por persona las diferentes métricas en otro df para luego cruzar con la base original
filas = ['Persona','Rango_Cons']
columnas = ['Stock','Diabetes', 'Hepatitis', 'PMI','Cronico', 'Oncologico', 'GC', 'Discapacidad', 'algun_diag']
aux_diag = pd.DataFrame(dfs_ss.pivot_table(columnas , filas, aggfunc = np.sum))

# Genero una columna que calcule el % de veces que se tuvo algún diagnóstico:
aux_diag['prop_diag'] = aux_diag['algun_diag'] / aux_diag['Stock']
aux_diag.tail()

# Creamos la marca a cada persona
# Analizando las diferentes posibilidades llegué a la conclusión que si la proporción es mayor a 40% se debe poner "Con Diagnóstico"
aux_diag['Marca_Diagnostico'] = np.where(aux_diag['prop_diag'] > prop_limite, 'Con Diagnostico','Sin Diagnostico')

# Reseteo los multi index:
aux_diag = aux_diag.reset_index()

In [26]:
# Exporto el df a csv
aux_diag.to_csv(archivo_diagnosticos, encoding = 'latin1',  decimal=',')